In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
import joblib
import warnings
import pickle
warnings.simplefilter("ignore")

In [12]:
df = pd.read_excel("C:/workspace/Reto/Datos/df_reducido.xlsx")

In [40]:
df.columns

Index(['ID_Reserva', 'Fecha_hoy', 'h_res_fec', 'h_num_per', 'aa_h_num_per',
       'h_num_adu', 'aa_h_num_adu', 'h_num_men', 'aa_h_num_men', 'h_num_noc',
       'aa_h_num_noc', 'h_tot_hab', 'aa_h_tot_hab', 'ID_Programa',
       'ID_Paquete', 'ID_Segmento_Comp', 'ID_canal', 'h_fec_lld', 'h_fec_reg',
       'h_fec_sda', 'ID_Pais_Origen', 'Reservacion', 'aa_Reservacion',
       'ID_estatus_reservaciones', 'h_can_res', 'h_edo', 'h_tfa_total',
       'aa_h_tfa_total', 'h_ult_cam_fec', 'dia_reservacion',
       'entre/fin_reservacion', 'ID_Agencia', 'ID_Tipo_Habitacion',
       'tarifa_x_noche', 'cluster'],
      dtype='object')

In [59]:
cols_class = ['h_num_adu', 'h_num_men', 'h_num_noc', 'h_edo', 'ID_Agencia',
              'ID_canal', 'h_can_res', 'entre/fin_reservacion']
# Dia, id de agencia, estado, h can reservación, h res fecha (solo mes), h fecha llegada(solo mes)

In [60]:
df_class = df[cols_class]

In [61]:
df_class['mes_reservacion'] = df['h_res_fec'].dt.strftime("%B")
df_class['mes_estadia'] = df['h_fec_lld'].dt.strftime("%B")

In [62]:
# Supón que 'df' es tu dataframe, y 'target' es la columna objetivo
X = df_class
y = df['cluster']

In [56]:
df_class.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95918 entries, 0 to 95917
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   h_num_adu              95918 non-null  int64 
 1   h_num_men              95918 non-null  int64 
 2   h_num_noc              95918 non-null  int64 
 3   h_edo                  95918 non-null  object
 4   ID_canal               95918 non-null  int64 
 5   h_can_res              95918 non-null  object
 6   entre/fin_reservacion  95918 non-null  object
 7   mes_reservacion        95918 non-null  object
 8   mes_estadia            95918 non-null  object
dtypes: int64(4), object(5)
memory usage: 6.6+ MB


In [63]:
# Codifica las variables categóricas
X_encoded = pd.get_dummies(X)

# Divide en train/test
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Entrena el árbol de decisión
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

# Evalúa
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.92      0.90      7986
           1       0.87      0.85      0.86      5534
           2       0.80      0.78      0.79      2785
           3       0.77      0.75      0.76      2879

    accuracy                           0.85     19184
   macro avg       0.83      0.82      0.83     19184
weighted avg       0.85      0.85      0.85     19184



In [65]:
columnas_entrenamiento = X_encoded.columns.tolist()

import pickle
with open('clasificador.pkl', 'wb') as f:
    pickle.dump({
        'modelo': clf,
        'columnas': columnas_entrenamiento
        }, f)

In [66]:
with open('clasificador.pkl', 'rb') as f:
    datos_guardados = pickle.load(f)

In [67]:
modelo = datos_guardados['modelo']
columnas_entrenamiento = datos_guardados['columnas']

In [71]:
# Supón que estas eran las variables categóricas originales
nuevo_registro = pd.DataFrame([{
    'h_num_adu' : 2,
    'h_num_men': 1, 
    'h_num_noc': 7, 
    'h_edo': 'EMX  ', 
    'ID_canal': 1, 
    'h_can_res': '09',
    'entre/fin_reservacion': 'Entre semana', 
    'mes_reservacion': 'March', 
    'mes_estadia': 'April'
}])

# Codificarlo igual que durante el entrenamiento
nuevo_encoded = pd.get_dummies(nuevo_registro)

# Añadir columnas faltantes con 0
for col in columnas_entrenamiento:
    if col not in nuevo_encoded.columns:
        nuevo_encoded[col] = 0

# Reordenar para que coincida exactamente
nuevo_encoded = nuevo_encoded[columnas_entrenamiento]

In [72]:
prediccion = modelo.predict(nuevo_encoded)
print("Predicción:", prediccion[0])

Predicción: 3
